In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
df = pd.read_csv('data/anime_planet_full_details_cleaned.csv')
df.head() 

,title,alt_title,type,year,rating,genres,source,description,img_url,page,type_only,episodes
0,Frieren: Beyond Journey's End,Sousou no Frieren,TV (28 eps),2023 - 2024,4.6,"['Adventure', 'Fantasy', 'Shounen', 'Death of ...",VIZ Media,Elf mage Frieren and her courageous fellow adv...,https://cdn.anime-planet.com/anime/primary/fri...,1,TV,28
1,The Apothecary Diaries,Kusuriya no Hitorigoto,TV (24 eps),2023 - 2024,4.6,"['Drama', 'Mystery', 'Seinen', 'Slice of Life'...",J-Novel Club,"In the East is a land ruled by an emperor, who...",https://cdn.anime-planet.com/anime/primary/the...,1,TV,24
2,The Apothecary Diaries 2nd Season,Kusuriya no Hitorigoto 2nd Season,TV (24 eps),2025,4.6,"['Drama', 'Mystery', 'Seinen', 'Slice of Life'...",NaN,The second season of The Apothecary Diaries.,https://cdn.anime-planet.com/anime/primary/the...,1,TV,24
3,Fullmetal Alchemist: Brotherhood,Hagane no Renkinjutsushi: Full Metal Alchemist,TV (64 eps),2009 - 2010,4.6,"['Action', 'Adventure', 'Drama', 'Fantasy', 'M...",NaN,The foundation of alchemy is based on the law ...,https://cdn.anime-planet.com/anime/primary/ful...,1,TV,64
4,Heaven Official's Blessing 2nd Season,Tian Guan Ci Fu 2nd Season,Web (12 eps),2023 - 2024,4.5,"['BL', 'Romance', 'Shounen-ai', 'Afterlife', '...",NaN,When Xie Lian is sent on a mission to Ghost Ci...,https://cdn.anime-planet.com/anime/primary/hea...,1,Web,12


In [8]:
def safe_genres(x):
    try:
        if pd.isna(x):
            return ""
        return ' '.join(ast.literal_eval(x))
    except (ValueError, SyntaxError):
        return ""

df['genres'] = df['genres'].apply(safe_genres)

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer


# Vectorize genres
vectorizer = CountVectorizer()
genre_matrix = vectorizer.fit_transform(df['genres'])


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(genre_matrix)

In [ ]:
def recommend_anime(title, df, similarity_matrix, top_n=5):
    idx = df[df['title'].str.lower() == title.lower()].index
    if len(idx) == 0:
        return "Anime not found."
    idx = idx[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    recommended = [df.iloc[i[0]]['title'] for i in sim_scores[1:top_n+1]]
    return recommended

# Example usage:
recommend_anime("", df, similarity_matrix)

'Anime not found.'

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer


df['description'] = df['description'].fillna("")


tfidf = TfidfVectorizer(stop_words='english')
desc_matrix = tfidf.fit_transform(df['description'])


desc_similarity = cosine_similarity(desc_matrix)


def recommend_anime_by_description(title, df, desc_similarity, top_n=5):
    idx = df[df['title'].str.lower() == title.lower()].index
    if len(idx) == 0:
        return "Anime not found."
    idx = idx[0]
    sim_scores = list(enumerate(desc_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    recommended = [df.iloc[i[0]]['title'] for i in sim_scores[1:top_n+1]]
    return recommended

recommend_anime_by_description("Naruto", df, desc_similarity)

['Naruto Shippuden',
 'Boruto: Naruto the Movie',
 'Naruto Special 2: Battle at Hidden Falls. I am the Hero!',
 'Naruto: Hidden Leaf Village Grand Sports Festival!',
 'Naruto Shippuden Movie 4: The Lost Tower']